# M3 - Imagen

In [1]:
import cv2, json, math, numpy as np, pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

IMG_PATH = "./P22C-ML_0_calibrated.png"
OUT_OVERLAY = "./sole_overlay_mm_v2.png"
OUT_MASK = "./sole_mask_v2.png"
OUT_CSV = "./sole_measurements_v2.csv"
SCALE_JSON = "./calibracion_px_mm.json"

In [6]:
# 
"""
Pipeline de segmentación (OpenCV + GrabCut) y metrología de lenguado.
Incluye:
  - Prefiltrado (NLMeans + bilateral + CLAHE)
  - Umbral dinámico del azul (HSV) para detectar la cinta y anclar la ROI
  - Semillas reforzadas para GrabCut (BG seguro = azul; PR_FG = no-azul dilatado;
    FG seguro = alto contraste H/S respecto al azul + distance transform)
  - Postproceso morfológico orientado (cierre/apertura)
  - Medición por PCA (longitud, anchura) y área desde el contorno
  - Modo debug (guarda artefactos intermedios)
  - Conversión opcional a mm (get_mm_per_px() o scale.json)

Referencias clave (DOI):
- GrabCut / Graph Cuts: Rother et al. 2004 (10.1145/1015706.1015720), Boykov & Jolly 2001 (10.1109/ICCV.2001.937505)
- NLMeans: Buades et al. 2005 (10.1109/CVPR.2005.38)
- Bilateral: Tomasi & Manduchi 1998 (10.1109/ICCV.1998.710815)
- CLAHE: Zuiderveld 1994 (10.1016/B978-0-12-336156-1.50061-6)
- PCA: Jolliffe & Cadima 2016 (10.1098/rsta.2015.0202)
"""

import cv2
import json
import numpy as np
import pandas as pd
from pathlib import Path

# =========================
# Configuración del pipeline
# =========================
CONFIG = {
    "img_path": "./P22C-ML_0_calibrated.png",
    "out_prefix": "./out/sole_",     # prefijo para salidas (debug/overlay/máscara/csv)
    "pad_roi": 12,                       # padding de la ROI (px)
    # Prefiltrado
    "nlmeans_h": 5,
    "bilateral_d": 9, "bilateral_sigmaC": 50, "bilateral_sigmaS": 50,
    "clahe_clip": 2.0, "clahe_tiles": (8, 8),
    # Umbral dinámico azul
    "blue_sat_min": 80,       # mínimo S para estimar el pico de H
    "blue_half_window": 15,   # ±ΔH alrededor del pico de H
    # Semillas
    "pr_fg_kernel": (7, 19),          # dilatación anisotrópica (W,H) para PR_FG
    "sure_fg_pct_contrast": 70,       # percentil del mapa de contraste H/S
    "sure_fg_pct_dt": 60,             # percentil de distance transform para reforzar FG
    "gc_iters": 12,                   # iteraciones de GrabCut
    # Postproceso
    "post_close_kernel": (9, 13),
    # Métrica (opcional)
    "scale_json": "./calibracion_px_mm.json",  # {"mm_per_px": 0.1234}
    # Debug
    "debug": True
}

# =========================
# Utilidades
# =========================
def save_img(name: str, img) -> str:
    """Guarda una imagen (BGR o 1 canal) con el prefijo de salida y devuelve su ruta."""
    p = CONFIG["out_prefix"] + name
    if img.ndim == 2:  # gris
        cv2.imwrite(p, img)
    else:
        cv2.imwrite(p, img if img.shape[2] == 3 else img[..., :3])
    return p

def get_scale_mm_per_px() -> float | None:
    """
    Intenta obtener mm/px desde:
    1) función get_mm_per_px() del usuario (si existe),
    2) archivo JSON CONFIG['scale_json'] con {"mm_per_px": valor}.
    """
    # 1) Función del usuario
    try:
        from get_mm_per_px import get_mm_per_px  # type: ignore
        val = float(get_mm_per_px())
        if val > 0:
            return val
    except Exception:
        pass
    # 2) JSON
    p = Path(CONFIG["scale_json"])
    if p.exists():
        try:
            data = json.loads(p.read_text())
            val = float(data.get("mm_per_px", 0))
            if val > 0:
                return val
        except Exception:
            pass
    return None

# =========================
# Bloques del pipeline
# =========================
def prefilter(bgr: np.ndarray) -> np.ndarray:
    """NLMeans + bilateral + CLAHE(V) → BGR prefiltrado."""
    den = cv2.fastNlMeansDenoisingColored(
        bgr, None,
        h=CONFIG["nlmeans_h"], hColor=CONFIG["nlmeans_h"],
        templateWindowSize=7, searchWindowSize=21
    )
    den = cv2.bilateralFilter(
        den, d=CONFIG["bilateral_d"],
        sigmaColor=CONFIG["bilateral_sigmaC"], sigmaSpace=CONFIG["bilateral_sigmaS"]
    )
    hsv = cv2.cvtColor(den, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)
    clahe = cv2.createCLAHE(CONFIG["clahe_clip"], CONFIG["clahe_tiles"])
    v = clahe.apply(v)
    out = cv2.cvtColor(cv2.merge([h, s, v]), cv2.COLOR_HSV2BGR)
    if CONFIG["debug"]:
        save_img("01_prefilter.png", out)
    return out

def dynamic_blue_threshold(hsv: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    """Calcula [lower, upper] HSV del azul a partir del pico de H en banda central con S>umbral."""
    H, S, _ = cv2.split(hsv)
    Hh, Wh = H.shape
    x1, x2 = int(Wh * 0.32), int(Wh * 0.68)  # banda central para evitar bordes metálicos
    band = H[:, x1:x2]
    band_s = S[:, x1:x2]
    mask_sat = band_s > CONFIG["blue_sat_min"]
    hh = band[mask_sat].flatten()
    if hh.size == 0:
        lower = np.array([95, 100, 30], np.uint8)
        upper = np.array([125, 255, 255], np.uint8)
    else:
        hist = cv2.calcHist([hh.astype(np.uint8)], [0], None, [180], [0, 180]).flatten()
        peak = int(np.argmax(hist))
        hw = CONFIG["blue_half_window"]
        lower = np.array([max(0, peak - hw), 80, 30], np.uint8)
        upper = np.array([min(179, peak + hw), 255, 255], np.uint8)
    return lower, upper

def belt_roi_from_blue(hsv: np.ndarray, lower: np.ndarray, upper: np.ndarray) -> tuple[tuple[int,int,int,int], np.ndarray]:
    """Umbraliza el azul, limpia (open/close) y devuelve (x,y,w,h) de la mayor componente + máscara azul."""
    blue = cv2.inRange(hsv, lower, upper)
    k = cv2.getStructuringElement(cv2.MORPH_RECT, (9, 9))
    blue = cv2.morphologyEx(blue, cv2.MORPH_OPEN, k, iterations=1)
    blue = cv2.morphologyEx(blue, cv2.MORPH_CLOSE, k, iterations=2)
    cnts, _ = cv2.findContours(blue, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not cnts:
        raise RuntimeError("No se detectó la cinta azul")
    c = max(cnts, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(c)
    H, W = blue.shape
    pad = CONFIG["pad_roi"]
    x = max(0, x - pad); y = max(0, y - pad)
    w = min(W - x, w + 2 * pad); h = min(H - y, h + 2 * pad)
    if CONFIG["debug"]:
        save_img("02_blue_mask.png", blue)
        dbg = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
        cv2.rectangle(dbg, (x, y), (x + w, y + h), (255, 0, 255), 3)
        save_img("03_roi_debug.png", dbg)
    return (x, y, w, h), blue

def seeds_in_roi(bgr_roi: np.ndarray, lower: np.ndarray, upper: np.ndarray) -> np.ndarray:
    """
    Genera máscara de estados para GrabCut (gc_mask):
      - BG seguro: azul.
      - PR_FG: no-azul dilatado anisotrópicamente (conectar cabeza/cola).
      - FG seguro: no-azul con alto contraste H/S respecto al azul + refuerzo por DT.
    """
    hsv = cv2.cvtColor(bgr_roi, cv2.COLOR_BGR2HSV)
    blue = cv2.inRange(hsv, lower, upper)
    not_blue = cv2.bitwise_not(blue)

    # PR_FG (no-azul dilatado verticalmente)
    k_vert = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, CONFIG["pr_fg_kernel"])
    pr_fg = cv2.dilate(not_blue, k_vert, iterations=1)

    # FG seguro: contraste H/S + distance transform
    h, s, _ = cv2.split(hsv)
    belt_h = h[blue > 0].astype(np.float32)
    belt_s = s[blue > 0].astype(np.float32)
    if belt_h.size:
        mu_h, mu_s = float(np.median(belt_h)), float(np.median(belt_s))
        dh = cv2.absdiff(h.astype(np.int16), np.full_like(h, int(mu_h), np.int16)).astype(np.uint8)
        ds = cv2.absdiff(s.astype(np.int16), np.full_like(s, int(mu_s), np.int16)).astype(np.uint8)
        contrast = cv2.addWeighted(dh, 0.7, ds, 0.3, 0)
        thr = int(np.percentile(contrast, CONFIG["sure_fg_pct_contrast"]))
        _, hi = cv2.threshold(contrast, thr, 255, cv2.THRESH_BINARY)
        sure_fg = cv2.bitwise_and(hi, not_blue)
        if CONFIG["debug"]:
            norm_c = contrast.copy()
            cv2.normalize(norm_c, norm_c, 0, 255, cv2.NORM_MINMAX)
            save_img("07_contrast.png", norm_c)
    else:
        sure_fg = cv2.erode(pr_fg, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (9, 9)), 1)

    dist = cv2.distanceTransform((pr_fg > 0).astype(np.uint8), cv2.DIST_L2, 5)
    if np.any(dist > 0):
        th = np.percentile(dist[dist > 0], CONFIG["sure_fg_pct_dt"])
        sure_fg |= (dist > th).astype(np.uint8) * 255

    # Montaje de gc_mask
    m = 6  # borde ROI como BG
    gc_mask = np.full(blue.shape, cv2.GC_PR_BGD, np.uint8)
    gc_mask[blue > 0] = cv2.GC_BGD
    gc_mask[pr_fg > 0] = cv2.GC_PR_FGD
    gc_mask[sure_fg > 0] = cv2.GC_FGD
    gc_mask[:m, :] = cv2.GC_BGD; gc_mask[-m:, :] = cv2.GC_BGD
    gc_mask[:, :m] = cv2.GC_BGD; gc_mask[:, -m:] = cv2.GC_BGD

    if CONFIG["debug"]:
        save_img("04_pr_fg.png", pr_fg)
        save_img("05_sure_fg.png", sure_fg)
        save_img("06_not_blue.png", not_blue)
        save_img("08_gc_seeds.png", (gc_mask * (255 // 3)).astype(np.uint8))
    return gc_mask

# =========================
# Orquestador
# =========================
def run():
    IMG_PATH = CONFIG["img_path"]
    OUT_OVERLAY = CONFIG["out_prefix"] + "overlay_debug.png"
    OUT_MASK = CONFIG["out_prefix"] + "mask_debug.png"
    OUT_CSV = CONFIG["out_prefix"] + "measurements_debug.csv"

    # 1) Carga + prefiltrado
    bgr0 = cv2.imread(IMG_PATH, cv2.IMREAD_COLOR)
    if bgr0 is None:
        raise FileNotFoundError(f"No se pudo leer la imagen: {IMG_PATH}")
    bgr = prefilter(bgr0)

    # 2) Umbral azul dinámico y ROI de la cinta
    hsv = cv2.cvtColor(bgr, cv2.COLOR_BGR2HSV)
    lower, upper = dynamic_blue_threshold(hsv)
    roi, _ = belt_roi_from_blue(hsv, lower, upper)
    x, y, w, h = roi
    crop = bgr[y:y+h, x:x+w].copy()

    # 3) Semillas + GrabCut + postproceso
    gc_mask = seeds_in_roi(crop, lower, upper)
    bg_model = np.zeros((1, 65), np.float64)
    fg_model = np.zeros((1, 65), np.float64)
    cv2.grabCut(crop, gc_mask, None, bg_model, fg_model, CONFIG["gc_iters"], cv2.GC_INIT_WITH_MASK)
    mask_roi = np.where((gc_mask == cv2.GC_FGD) | (gc_mask == cv2.GC_PR_FGD), 1, 0).astype(np.uint8)
    if CONFIG["debug"]:
        save_img("09_mask_roi_raw.png", (mask_roi * 255))

    k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, CONFIG["post_close_kernel"])
    mask_roi = cv2.morphologyEx(mask_roi, cv2.MORPH_CLOSE, k, iterations=1)
    mask_roi = cv2.morphologyEx(mask_roi, cv2.MORPH_OPEN,
                                cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5)), iterations=1)

    full = np.zeros(bgr.shape[:2], np.uint8)
    full[y:y+h, x:x+w] = mask_roi

    # 4) Selección de contorno del pez (área + alargamiento)
    cnts, _ = cv2.findContours((full * 255).astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    if not cnts:
        raise RuntimeError("No contornos tras segmentación.")

    def score(c):
        area = cv2.contourArea(c)
        if area < 1500:
            return -1
        (w_, h_) = cv2.minAreaRect(c)[1]
        if w_ == 0 or h_ == 0:
            return -1
        aspect = max(w_, h_) / max(1.0, min(w_, h_))
        return area + 3500.0 * max(0.0, aspect - 1.3)

    cnt = max(cnts, key=score)

    # 5) Medición por PCA + área
    pts = cnt.reshape(-1, 2).astype(np.float32)
    mean, eigvecs, _ = cv2.PCACompute2(pts, mean=None)
    mean = mean.flatten()
    R = np.asarray(eigvecs, np.float32)
    pts_c = pts - mean
    pts_r = pts_c @ R.T
    min_x, max_x = np.min(pts_r[:, 0]), np.max(pts_r[:, 0])
    min_y, max_y = np.min(pts_r[:, 1]), np.max(pts_r[:, 1])
    length_px = float(max_x - min_x)
    width_px = float(max_y - min_y)
    area_px2 = float(cv2.contourArea(cnt))

    # 6) Overlay (perímetro + caja PCA + eje mayor + ROI)
    rgb = cv2.cvtColor(bgr0, cv2.COLOR_BGR2RGB)
    overlay = rgb.copy()
    box_r = np.array([[min_x, min_y], [max_x, min_y], [max_x, max_y], [min_x, max_y]], np.float32)
    box_img = (box_r @ R) + mean
    box_img = np.int32(box_img)
    cv2.drawContours(overlay, [cnt], -1, (0, 255, 0), 3)
    cv2.polylines(overlay, [box_img], True, (255, 255, 0), 2)
    i_max = np.argmax(pts_r[:, 0]); i_min = np.argmin(pts_r[:, 0])
    p1 = (np.array([pts_r[i_min, 0], 0.0], np.float32) @ R) + mean
    p2 = (np.array([pts_r[i_max, 0], 0.0], np.float32) @ R) + mean
    cv2.line(overlay, tuple(np.int32(p1)), tuple(np.int32(p2)), (255, 0, 0), 2)
    cv2.rectangle(overlay, (x, y), (x + w, y + h), (255, 0, 255), 2)

    # 7) Conversión opcional a mm
    mm_per_px = get_scale_mm_per_px()
    if mm_per_px is not None and mm_per_px > 0:
        length_mm = length_px * mm_per_px
        width_mm = width_px * mm_per_px
        area_mm2 = area_px2 * (mm_per_px ** 2)
    else:
        length_mm = width_mm = area_mm2 = float("nan")

    # 8) Guardado de salidas
    mask_path = save_img("mask_debug.png", (full * 255))
    overlay_path = save_img("overlay_debug.png", cv2.cvtColor(overlay, cv2.COLOR_RGB2BGR))

    df = pd.DataFrame([{
        "image_path": IMG_PATH,
        "roi_x": x, "roi_y": y, "roi_w": w, "roi_h": h,
        "length_px": length_px, "width_px": width_px, "area_px2": area_px2,
        "mm_per_px": (float(mm_per_px) if mm_per_px else float("nan")),
        "length_mm": float(length_mm), "width_mm": float(width_mm), "area_mm2": float(area_mm2)
    }])
    csv_path = CONFIG["out_prefix"] + "measurements_debug.csv"
    df.to_csv(csv_path, index=False)

    return overlay_path, mask_path, csv_path, length_px, width_px, area_px2, mm_per_px, (x, y, w, h)

In [5]:
# === Ejecutar ===
overlay_p, mask_p, csv_p, Lpx, Wpx, Apx2, mmpp, roi = run()
print(f"[OK] ROI: {roi}")
print(f"[OK] L={Lpx:.1f}px  W={Wpx:.1f}px  A={Apx2:.1f}px^2")
if mmpp:
    print(f"[OK] mm/px={mmpp}  L={Lpx*mmpp:.2f} mm  W={Wpx*mmpp:.2f} mm  A={Apx2*(mmpp**2):.2f} mm^2")
print(f"[OK] Overlay:   {overlay_p}")
print(f"[OK] Máscara:   {mask_p}")
print(f"[OK] CSV:       {csv_p}")

[OK] ROI: (9, 0, 1119, 1080)
[OK] L=745.1px  W=294.3px  A=152294.5px^2
[OK] mm/px=0.05  L=37.26 mm  W=14.72 mm  A=380.74 mm^2
[OK] Overlay:   ./out/sole_overlay_debug.png
[OK] Máscara:   ./out/sole_mask_debug.png
[OK] CSV:       ./out/sole_measurements_debug.csv
